# Hands On Tutorial 3: Multitask Deep Learning for Drug Discovery

Implemntation of the model described in https://arxiv.org/pdf/1502.02072.pdf

### Imports and parameters

In [39]:
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Conv1D, LSTM, Dropout
from keras.optimizers import SGD

import pandas as pd
import numpy as np
import zipfile
import urllib

### Dataset preparation

We will use the QM9 dataset: http://quantum-machine.org/datasets/
* L. Ruddigkeit, R. van Deursen, L. C. Blum, J.-L. Reymond, Enumeration of 166 billion organic small molecules in the chemical universe database GDB-17, J. Chem. Inf. Model. 52, 2864–2875, 2012.
* R. Ramakrishnan, P. O. Dral, M. Rupp, O. A. von Lilienfeld, Quantum chemistry structures and properties of 134 kilo molecules, Scientific Data 1, 140022, 2014. [bibtex]

In [40]:
# download and extract dataset
url = ''
urllib.request.urlretrieve(url, 'qm9/qm9.pkl.zip')

with zipfile.open('qm9/qm9.pkl.zip', 'r') as file:
    data = pd.read_pickle(file)

ValueError: unknown url type: ''

In [27]:
# preprocess data
labels = ['A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'U0', 'U', 'H', 'G', 'Cv']

properties = {}
for i, key in enumerate(labels):
    properties[key] = data['properties'][:, i]
    
properties['smiles'] = data['smiles_optimized']

df = pd.DataFrame(properties)
df = df[['smiles'] + labels]
df.head()

#### Task 1:
convert smiles strings into one-hot representation

### Building Model Architecture

### Compile model and set loss function and metrics


### Model Training


### Model Evaluation
